In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import csv

train_data = pd.read_csv('train_trendency.csv')
train_data = train_data.drop(['Unnamed: 0'], axis=1)

states = train_data.Province_State.unique()
scalers = dict()

state_data = dict()
for state in states:
    scalers[state] = MinMaxScaler(feature_range=(-1,1))
    scaler = scalers[state]
    x = train_data.copy()
    state_data[state] = x[x.Province_State.isin([state])]
    state_data[state] = state_data[state].drop(['Recovered', 'Active', 'Province_State', 'Date'], axis=1)
    print(state_data[state].head())
    state_data[state] = scaler.fit_transform(state_data[state])
    print(state_data[state][0])

test_size = 30
test_data = dict()
train_data = dict()
for state in states:
    train_data[state] = torch.FloatTensor(state_data[state][:-test_size])
    test_data[state] = state_data[state][-test_size:]
    
print(len(state_data['California']))
print(len(train_data['California']))
print(len(test_data['California']))

state_data['Wyoming']

# Shape of each of these is (79, 6)

     Confirmed  Deaths  Incident_Rate  Total_Test_Results  \
0       407848    5573    8318.021857           1981019.0   
50      410995    5760    8382.204628           1989724.0   
100     414583    5945    8455.381553           1999693.0   
150     417528    6030    8515.444553           2012213.0   
200     420681    6119    8579.749693           2021123.0   

     Case_Fatality_Ratio  Testing_Rate  
0               1.366440  40402.697430  
50              1.401477  40580.235092  
100             1.433971  40783.551916  
150             1.444215  41038.896146  
200             1.454546  41220.614764  
[-1. -1. -1. -1. -1. -1.]
     Confirmed  Deaths  Incident_Rate  Total_Test_Results  \
1        50544     225    6906.205360           1357708.0   
51       50840     226    6946.394275           1370835.0   
101      51185     228    6993.007949           1382276.0   
151      51485     229    7035.247319           1389226.0   
201      51678     229    7060.809656           1398649.

     Confirmed  Deaths  Incident_Rate  Total_Test_Results  \
37      737994   18045    5764.682347           8198497.0   
87      746002   18412    5827.235127           8255043.0   
137     753164   18710    5883.179559           8307622.0   
187     759243   18935    5930.664368           8368848.0   
237     765489   19143    5979.453661           8433901.0   

     Case_Fatality_Ratio  Testing_Rate  
37              2.445142  32312.080568  
87              2.468090  32477.461120  
137             2.484187  32624.141452  
187             2.493931  32769.165791  
237             2.500754  32940.256393  
[-1.         -1.         -1.         -1.         -0.92238916 -1.        ]
     Confirmed  Deaths  Incident_Rate  Total_Test_Results  \
38      101393    2109    9571.147135           2179821.0   
88      102485    2120    9674.228143           2199630.0   
138     103386    2130    9759.279415           2219805.0   
188     104443    2146    9859.056544           2242295.0   
238     

array([[-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -6.54485606e-01, -1.00000000e+00],
       [-9.47317310e-01, -1.00000000e+00, -9.47317310e-01,
        -9.80932178e-01, -7.01661556e-01, -9.80932178e-01],
       [-8.47293032e-01, -1.00000000e+00, -8.47293032e-01,
        -9.25980351e-01, -7.90073808e-01, -9.25980351e-01],
       [-7.96795339e-01, -1.00000000e+00, -7.96795339e-01,
        -8.37230248e-01, -8.34143295e-01, -8.37230248e-01],
       [-7.58193736e-01, -1.00000000e+00, -7.58193736e-01,
        -8.37230248e-01, -8.67579064e-01, -8.37230248e-01],
       [-6.86574411e-01, -1.00000000e+00, -6.86574411e-01,
        -8.37230248e-01, -9.29043435e-01, -8.37230248e-01],
       [-6.02816218e-01, -1.00000000e+00, -6.02816218e-01,
        -8.37230248e-01, -1.00000000e+00, -8.37230248e-01],
       [-5.50861860e-01, -6.85393258e-01, -5.50861860e-01,
        -6.80421412e-01, -5.01274146e-01, -6.80421412e-01],
       [-5.01820830e-01, -6.85393258e-01, -5.018

In [2]:
train_window = 30
# From https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

seqs = dict()
for state in states:
    seqs[state] = create_inout_sequences(train_data[state], train_window)

In [3]:
# Also from https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python
class LSTM(nn.Module):
    def __init__(self, input_size=6, hidden_layer_size=100, output_size=6):
        # input needs to be 6 for 6 columns of train data
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [4]:
models = dict()
optimizers = dict()
for state in states:
    models[state] = LSTM()
    optimizers[state] = torch.optim.Adam(models[state].parameters(), lr=0.0001)

loss_function = nn.MSELoss()

In [5]:
epochs = 26

for state in states:
    
    train_inout_seq = seqs[state]
    model = models[state]
    optimizer = optimizers[state]
    
    for i in range(epochs):
        for seq, labels in train_inout_seq:
            
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(seq)
            #print('y_pred', y_pred, 'labels', labels)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        if i%25 == 1:
            print(f'state: {state} epoch: {i:3} loss: {single_loss.item():10.8f}')

    print(f'state: {state} epoch: {i:3} loss: {single_loss.item():10.10f}')

state: Alabama epoch:   1 loss: 0.38073564
state: Alabama epoch:  25 loss: 0.0021268607
state: Alaska epoch:   1 loss: 0.26956618
state: Alaska epoch:  25 loss: 0.0172842424
state: Arizona epoch:   1 loss: 0.37493742
state: Arizona epoch:  25 loss: 0.0050660013
state: Arkansas epoch:   1 loss: 0.31887218
state: Arkansas epoch:  25 loss: 0.2119907141
state: California epoch:   1 loss: 0.35035583
state: California epoch:  25 loss: 0.0073399935
state: Colorado epoch:   1 loss: 0.10720878
state: Colorado epoch:  25 loss: 0.0013679699
state: Connecticut epoch:   1 loss: 0.11842328
state: Connecticut epoch:  25 loss: 0.0012384824
state: Delaware epoch:   1 loss: 0.18938948
state: Delaware epoch:  25 loss: 0.0074357693
state: Florida epoch:   1 loss: 0.28776118
state: Florida epoch:  25 loss: 0.0017231781
state: Georgia epoch:   1 loss: 0.25699794
state: Georgia epoch:  25 loss: 0.0018825982
state: Hawaii epoch:   1 loss: 0.23633313
state: Hawaii epoch:  25 loss: 0.0157086998
state: Idaho epo

In [6]:
model = models['California']
seq, label = seqs['California'][0]

print(label.tolist())
model(seq).tolist()

[0.5051944851875305, 0.08346163481473923, 0.5208973288536072, -0.020106134936213493, -0.13405448198318481, -0.020106134936213493]


[0.4377366304397583,
 0.20103217661380768,
 0.48117637634277344,
 0.10968299210071564,
 0.035322077572345734,
 0.09772939234972]

In [7]:
state_predictions = dict()
actual_values = dict()
for state in states:
    print(state)
    model = models[state]
    state_predictions[state] = test_data[state].tolist()

    pred = state_predictions[state]
    for _ in range(30):
        pred.append(model(torch.FloatTensor(pred[-30:])).tolist())
    scaler = scalers[state]
    
    actual_values[state] = scaler.inverse_transform(pred[30:])
    print(actual_values[state])

Alabama
[[5.01168657e+05 1.04467335e+04 1.02575552e+04 2.30778860e+06
  2.12915180e+00 4.72881313e+04]
 [5.00869998e+05 1.04688427e+04 1.02548775e+04 2.30867091e+06
  2.12536213e+00 4.72830330e+04]
 [5.00590521e+05 1.04764609e+04 1.02515267e+04 2.30884240e+06
  2.12256821e+00 4.72579718e+04]
 [5.00393654e+05 1.04783325e+04 1.02496423e+04 2.30879548e+06
  2.12054337e+00 4.72387347e+04]
 [5.00239612e+05 1.04770585e+04 1.02482268e+04 2.30865198e+06
  2.11895011e+00 4.72234625e+04]
 [5.00113158e+05 1.04740899e+04 1.02469440e+04 2.30846342e+06
  2.11763710e+00 4.72108371e+04]
 [5.00004824e+05 1.04702591e+04 1.02456735e+04 2.30825298e+06
  2.11651414e+00 4.72001154e+04]
 [4.99909343e+05 1.04660697e+04 1.02443847e+04 2.30803420e+06
  2.11553089e+00 4.71908555e+04]
 [4.99823531e+05 1.04618013e+04 1.02430794e+04 2.30781456e+06
  2.11465497e+00 4.71827451e+04]
 [4.99745557e+05 1.04576241e+04 1.02417761e+04 2.30759912e+06
  2.11386704e+00 4.71755730e+04]
 [4.99674321e+05 1.04536273e+04 1.02404942

[[3.23830476e+05 5.47945999e+03 1.08065612e+04 2.63645116e+06
  1.69497963e+00 8.79808955e+04]
 [3.23604777e+05 5.47545543e+03 1.07861004e+04 2.63744119e+06
  1.69467921e+00 8.78801646e+04]
 [3.23409508e+05 5.47040503e+03 1.07734318e+04 2.63777691e+06
  1.69434854e+00 8.78154980e+04]
 [3.23223008e+05 5.46597993e+03 1.07631342e+04 2.63778131e+06
  1.69405428e+00 8.77664286e+04]
 [3.23046308e+05 5.46204377e+03 1.07543439e+04 2.63754566e+06
  1.69379844e+00 8.77251403e+04]
 [3.22883791e+05 5.45859326e+03 1.07467647e+04 2.63718394e+06
  1.69357596e+00 8.76892984e+04]
 [3.22736406e+05 5.45557510e+03 1.07401877e+04 2.63677151e+06
  1.69338082e+00 8.76580860e+04]
 [3.22603869e+05 5.45292965e+03 1.07344508e+04 2.63635173e+06
  1.69320816e+00 8.76309986e+04]
 [3.22485291e+05 5.45060535e+03 1.07294281e+04 2.63594823e+06
  1.69305452e+00 8.76075794e+04]
 [3.22379331e+05 5.44855523e+03 1.07250104e+04 2.63557226e+06
  1.69291709e+00 8.75873473e+04]
 [3.22284703e+05 5.44674131e+03 1.07211119e+04 2.6

[[9.26566740e+04 1.48794658e+03 9.50661749e+03 1.48331658e+06
  1.77065847e+00 1.52390752e+05]
 [9.27324696e+04 1.49039365e+03 9.50340577e+03 1.48249890e+06
  1.77295578e+00 1.52304978e+05]
 [9.27570782e+04 1.49132507e+03 9.50586923e+03 1.48254927e+06
  1.77409894e+00 1.52350764e+05]
 [9.27845414e+04 1.49213550e+03 9.50862498e+03 1.48285194e+06
  1.77518772e+00 1.52416316e+05]
 [9.28088796e+04 1.49277754e+03 9.51155708e+03 1.48318453e+06
  1.77607323e+00 1.52475208e+05]
 [9.28294352e+04 1.49330068e+03 9.51424236e+03 1.48350257e+06
  1.77678905e+00 1.52527267e+05]
 [9.28467694e+04 1.49373162e+03 9.51660341e+03 1.48378613e+06
  1.77737067e+00 1.52572146e+05]
 [9.28615954e+04 1.49409147e+03 9.51864727e+03 1.48403256e+06
  1.77784914e+00 1.52610494e+05]
 [9.28743467e+04 1.49439316e+03 9.52039641e+03 1.48424325e+06
  1.77824524e+00 1.52642994e+05]
 [9.28853825e+04 1.49464715e+03 9.52188702e+03 1.48442236e+06
  1.77857525e+00 1.52670431e+05]
 [9.28950383e+04 1.49486243e+03 9.52316171e+03 1.4

[[1.77573169e+05 1.94739539e+03 9.95526155e+03 6.60821659e+05
  1.11011811e+00 3.69700049e+04]
 [1.77803594e+05 1.94727379e+03 9.94881845e+03 6.60750055e+05
  1.11026509e+00 3.69839933e+04]
 [1.77837995e+05 1.94698393e+03 9.94712598e+03 6.60648675e+05
  1.11016253e+00 3.69800975e+04]
 [1.77863731e+05 1.94670634e+03 9.94641759e+03 6.60572874e+05
  1.11010817e+00 3.69777666e+04]
 [1.77885625e+05 1.94649370e+03 9.94597535e+03 6.60526567e+05
  1.11007255e+00 3.69762914e+04]
 [1.77902767e+05 1.94634046e+03 9.94569597e+03 6.60498837e+05
  1.11004928e+00 3.69753904e+04]
 [1.77915921e+05 1.94623183e+03 9.94551877e+03 6.60483224e+05
  1.11003449e+00 3.69748677e+04]
 [1.77925879e+05 1.94615437e+03 9.94540307e+03 6.60475041e+05
  1.11002530e+00 3.69745693e+04]
 [1.77933464e+05 1.94610028e+03 9.94533167e+03 6.60471749e+05
  1.11002002e+00 3.69744258e+04]
 [1.77939196e+05 1.94606246e+03 9.94528809e+03 6.60471179e+05
  1.11001721e+00 3.69743663e+04]
 [1.77943579e+05 1.94603691e+03 9.94526511e+03 6.6

[[3.03173847e+05 4.85578784e+03 1.04065004e+04 1.27555142e+06
  1.61862279e+00 4.36413481e+04]
 [3.02938599e+05 4.85295868e+03 1.03999161e+04 1.27572137e+06
  1.61826879e+00 4.36424210e+04]
 [3.02829365e+05 4.85055878e+03 1.03963415e+04 1.27550060e+06
  1.61761659e+00 4.36411383e+04]
 [3.02736688e+05 4.84865164e+03 1.03936288e+04 1.27529177e+06
  1.61711443e+00 4.36394277e+04]
 [3.02662779e+05 4.84712891e+03 1.03915443e+04 1.27510344e+06
  1.61668316e+00 4.36368886e+04]
 [3.02602415e+05 4.84584743e+03 1.03898409e+04 1.27493398e+06
  1.61631927e+00 4.36339978e+04]
 [3.02552453e+05 4.84473746e+03 1.03884118e+04 1.27478280e+06
  1.61601343e+00 4.36310190e+04]
 [3.02510683e+05 4.84376536e+03 1.03871956e+04 1.27464855e+06
  1.61575595e+00 4.36281221e+04]
 [3.02475461e+05 4.84291033e+03 1.03861500e+04 1.27452973e+06
  1.61553816e+00 4.36254005e+04]
 [3.02445553e+05 4.84215764e+03 1.03852463e+04 1.27442471e+06
  1.61535292e+00 4.36228994e+04]
 [3.02420034e+05 4.84149621e+03 1.03844630e+04 1.2

...

[[1.84365052e+06 5.10852510e+04 9.39317877e+03 4.39884207e+07
  2.72649793e+00 2.22308493e+05]
 [1.84553118e+06 5.11171990e+04 9.39765143e+03 4.40417795e+07
  2.72550072e+00 2.22338277e+05]
 [1.84584312e+06 5.11255687e+04 9.40027209e+03 4.40672599e+07
  2.72480420e+00 2.22330961e+05]
 [1.84618912e+06 5.11338166e+04 9.40264829e+03 4.40874203e+07
  2.72419541e+00 2.22345515e+05]
 [1.84651832e+06 5.11411981e+04 9.40469667e+03 4.41035593e+07
  2.72368107e+00 2.22367420e+05]
 [1.84681810e+06 5.11476569e+04 9.40644218e+03 4.41167032e+07
  2.72324872e+00 2.22391013e+05]
 [1.84708386e+06 5.11532421e+04 9.40791491e+03 4.41275355e+07
  2.72288681e+00 2.22413777e+05]
 [1.84731630e+06 5.11580479e+04 9.40915392e+03 4.41365400e+07
  2.72258251e+00 2.22434683e+05]
 [1.84751735e+06 5.11621584e+04 9.41019319e+03 4.41440498e+07
  2.72232635e+00 2.22453316e+05]
 [1.84768990e+06 5.11656647e+04 9.41106563e+03 4.41503255e+07
  2.72211007e+00 2.22469648e+05]
 [1.84783713e+06 5.11686491e+04 9.41179813e+03 4.4

[[4.46031491e+05 4.74451222e+03 1.13105687e+04 3.65620825e+06
  1.08169239e+00 9.19658029e+04]
 [4.46384930e+05 4.74584878e+03 1.13170874e+04 3.65693404e+06
  1.08032196e+00 9.19384210e+04]
 [4.46540047e+05 4.74470179e+03 1.13173040e+04 3.65567728e+06
  1.07961688e+00 9.19051838e+04]
 [4.46651768e+05 4.74374278e+03 1.13172110e+04 3.65480050e+06
  1.07915781e+00 9.18771378e+04]
 [4.46737017e+05 4.74303803e+03 1.13170749e+04 3.65412139e+06
  1.07881491e+00 9.18551240e+04]
 [4.46804486e+05 4.74251849e+03 1.13169892e+04 3.65361066e+06
  1.07855915e+00 9.18380298e+04]
 [4.46858425e+05 4.74212961e+03 1.13169584e+04 3.65322825e+06
  1.07836668e+00 9.18248165e+04]
 [4.46902100e+05 4.74184205e+03 1.13169818e+04 3.65294552e+06
  1.07822213e+00 9.18147005e+04]
 [4.46937987e+05 4.74163134e+03 1.13170499e+04 3.65273944e+06
  1.07811369e+00 9.18070162e+04]
 [4.46966789e+05 4.74146315e+03 1.13171301e+04 3.65258507e+06
  1.07803060e+00 9.18010927e+04]
 [4.46989844e+05 4.74132430e+03 1.13172119e+04 3.6

[[5.40864518e+05 9.01105647e+03 1.05535758e+04 5.23985373e+06
  1.67437949e+00 1.00634016e+05]
 [5.40380238e+05 9.01264372e+03 1.05529102e+04 5.23685474e+06
  1.67474480e+00 1.00652478e+05]
 [5.40117812e+05 9.01091868e+03 1.05534571e+04 5.23247149e+06
  1.67482676e+00 1.00654232e+05]
 [5.40004446e+05 9.00960690e+03 1.05539722e+04 5.22904900e+06
  1.67485562e+00 1.00650445e+05]
 [5.39959734e+05 9.00837533e+03 1.05542449e+04 5.22655495e+06
  1.67487388e+00 1.00645651e+05]
 [5.39950050e+05 9.00734001e+03 1.05543550e+04 5.22480458e+06
  1.67488784e+00 1.00641849e+05]
 [5.39955700e+05 9.00649252e+03 1.05543619e+04 5.22359897e+06
  1.67489915e+00 1.00639054e+05]
 [5.39968478e+05 9.00584069e+03 1.05543472e+04 5.22279365e+06
  1.67490919e+00 1.00637440e+05]
 [5.39983388e+05 9.00535349e+03 1.05543339e+04 5.22226529e+06
  1.67491804e+00 1.00636683e+05]
 [5.39997742e+05 9.00499438e+03 1.05543259e+04 5.22192228e+06
  1.67492555e+00 1.00636443e+05]
 [5.40009541e+05 9.00471596e+03 1.05543022e+04 5.2

[[3.83582955e+05 2.13316105e+03 1.18967461e+04 3.03102293e+06
  5.40413957e-01 9.41395599e+04]
 [3.83633581e+05 2.13244771e+03 1.18949288e+04 3.03159450e+06
  5.40236344e-01 9.41223990e+04]
 [3.83664100e+05 2.13145603e+03 1.18951928e+04 3.03138431e+06
  5.40192234e-01 9.41097624e+04]
 [3.83677266e+05 2.13092135e+03 1.18955417e+04 3.03133558e+06
  5.40166130e-01 9.41018482e+04]
 [3.83682467e+05 2.13057592e+03 1.18958264e+04 3.03128743e+06
  5.40148899e-01 9.40963910e+04]
 [3.83683237e+05 2.13034052e+03 1.18959956e+04 3.03124195e+06
  5.40136235e-01 9.40923399e+04]
 [3.83681787e+05 2.13017509e+03 1.18960682e+04 3.03119930e+06
  5.40126483e-01 9.40892363e+04]
 [3.83679336e+05 2.13005568e+03 1.18960754e+04 3.03116048e+06
  5.40118717e-01 9.40868035e+04]
 [3.83676486e+05 2.12996689e+03 1.18960405e+04 3.03112531e+06
  5.40112338e-01 9.40848520e+04]
 [3.83673505e+05 2.12989896e+03 1.18959803e+04 3.03109351e+06
  5.40106966e-01 9.40832572e+04]
 [3.83670583e+05 2.12984552e+03 1.18959087e+04 3.0

[[1.41196434e+05 2.59219858e+03 8.18969511e+03 2.57297521e+06
  1.79178296e+00 1.42424043e+05]
 [1.41372545e+05 2.59417763e+03 8.21575265e+03 2.57653623e+06
  1.79334987e+00 1.42659183e+05]
 [1.41451331e+05 2.59600003e+03 8.22661047e+03 2.57819928e+06
  1.79367207e+00 1.42844745e+05]
 [1.41519276e+05 2.59763683e+03 8.23632533e+03 2.57965052e+06
  1.79381275e+00 1.43013077e+05]
 [1.41574316e+05 2.59906562e+03 8.24436652e+03 2.58108265e+06
  1.79389469e+00 1.43151254e+05]
 [1.41618511e+05 2.60027651e+03 8.25104072e+03 2.58242773e+06
  1.79394179e+00 1.43264316e+05]
 [1.41654077e+05 2.60130179e+03 8.25659581e+03 2.58365936e+06
  1.79397108e+00 1.43356888e+05]
 [1.41682768e+05 2.60217320e+03 8.26123389e+03 2.58476979e+06
  1.79399207e+00 1.43433032e+05]
 [1.41705947e+05 2.60291353e+03 8.26510402e+03 2.58575430e+06
  1.79400927e+00 1.43495761e+05]
 [1.41724709e+05 2.60354367e+03 8.26834193e+03 2.58662022e+06
  1.79402488e+00 1.43547667e+05]
 [1.41739906e+05 2.60407979e+03 8.27105321e+03 2.5

In [8]:
csv_data = [["ID", "Confirmed", "Deaths"]]

count = 0

for i in range(30):
    for state in states:
        csv_data.append([count, int(actual_values[state][i][0]), int(actual_values[state][i][1])])
        count += 1

print(len(csv_data))

with open("submission.csv", "w+") as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerows(csv_data)


1501
